In [ ]:
import os
import googleapiclient.discovery
import googleapiclient.errors
from googleapiclient.http import MediaFileUpload
from google_auth_oauthlib.flow import InstalledAppFlow

In [ ]:
# Replace with your API key
API_KEY = os.getenv('YOUTUBE_API_KEY')
JSON_SECRETS_PATH = './secrets/oauth_secret.json'

# Replace with the path to the video you want to upload
VIDEO_PATH = 'f:/Jupyter/Manim/venv/media/videos/scene/480p15/DifferentRotations.mp4'

# Scope for YouTube Data API
SCOPES = ["https://www.googleapis.com/auth/youtube.upload"]

In [ ]:
def get_authenticated_service():
    credentials_file = 'credentials.pickle'

    # Check if credentials already exist
    if os.path.exists(credentials_file):
        with open(credentials_file, 'rb') as token:
            credentials = pickle.load(token)
    else:
        flow = InstalledAppFlow.from_client_secrets_file(JSON_SECRETS_PATH, SCOPES)
        credentials = flow.run_local_server()

        # Save the credentials for future use
        with open(credentials_file, 'wb') as token:
            pickle.dump(credentials, token)

    return googleapiclient.discovery.build("youtube", "v3", credentials=credentials)

def upload_video(youtube, video_path):
    # Prepare the video resource
    media = MediaFileUpload(video_path)

    # Set the video details
    body = {
        "snippet": {
            "title": "Your Video Title",
            "description": "Your Video Description",
            "tags": ["tag1", "tag2", "tag3"],
            "categoryId": "22"  # Replace with the appropriate category ID for your content
        },
        "status": {
            "privacyStatus": "private"  # Change to "public" if you want to make the video public
        }
    }

    try:
        # Execute the API request to upload the video
        youtube.videos().insert(part="snippet,status", body=body, media_body=media).execute()
        print("Video uploaded successfully.")
    except googleapiclient.errors.HttpError as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
if __name__ == "__main__":
    # Initialize the YouTube API client
    youtube = googleapiclient.discovery.build("youtube", "v3", developerKey=API_KEY)

    # Authenticate and get the authorized service
    # Uncomment the next line if running the script for the first time
    youtube = get_authenticated_service()

    # Upload the video
    upload_video(youtube, VIDEO_PATH)

<h1 align=center> Afresh </h1>

## Notes
On further perusing of the API reference I found out that

- The video resource can be used for managing videos on youtube.
    1. `insert` - Uploads a video to YouTube and optionally sets the video's metadata.
    2. `delete` - Deletes a YouTube video
    
Based on this we can attempt something basic


In [2]:
import os
from googleapiclient.discovery import build

In [3]:
from custom import extract_key_from_json

In [4]:
VIDEO_ID = input("Enter any video ID from the channel of interest: ")
API_KEY = os.getenv('YOUTUBE_API_KEY')

Enter any video ID from the channel of interest:  gU-8U7Z-E64


In [5]:
# Create a YouTube API client
youtube = build("youtube", "v3", developerKey=API_KEY)

In [6]:
response = youtube.videos().list(
                part="status",
                id=VIDEO_ID
            ).execute()

## Monitoring
We will need to monitor how the video is doing if we are going to automate this. We will require metrics for that. These are to be found in the JSON response as `status.<some_attribute>`

In [7]:
extract_key_from_json(response, "uploadStatus")

['processed']